In [ ]:
!nvidia-smi

Wed Nov 20 06:40:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!python -m pip install paddlepaddle-gpu==2.6.1.post120 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
!git clone https://github.com/PaddlePaddle/PaddleOCR

Looking in links: https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.8/796.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: opt-einsum
    Found existing installation: opt_einsum 3.4.0
    Uninstalling opt_einsum-3.4.0:
      Successfully uninstalled opt_einsum-3.4.0
Cloning into 'PaddleOCR'...
remote: Enumerating objects: 71829, done.
remote: Counting objects: 100% (20898/20898), done.
remote: Compressing objects: 100% (4242/4242), done.
remote: Total 71829 (delta 17188), reused 20157 (delta 16654), pack-reused 50931 (from 1)
Receiving objects: 100% (71829/71829), 485.62 MiB | 17.35 MiB/s, done.
Resolving deltas: 100% (52863/52863), done.
Updating files: 100% (1994/1994), done.


In [ ]:
%cd PaddleOCR
!pip3 install -r requirements.txt

/content/PaddleOCR
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 86.1 MB/s eta 0:00:00
  Attempting uninstall: albucore
    Found existing installation: albucore 0.0.19
    Uninstalling albucore-0.0.19:
      Successfully uninstalled albucore-0.0.19
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.20
    Uninstalling albumentations-1.4.20:
      Successfully uninstalled albumentations-1.4.20


In [ ]:
import os
import random
import pandas as pd

# Set the seed for reproducibility
random.seed(42)

# Path to the directory containing folders of images
base_dir = "the folder containing images that are annotated with their ground truths"

# Initialize lists to hold file paths and ground truth
image_paths = []
ground_truths = []

# Recursively read the directories and files
for root, _, files in os.walk(base_dir):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            file_path = os.path.join(root, file)
            file_name_without_ext = os.path.splitext(file)[0]
            image_paths.append(file_path)
            ground_truths.append(file_name_without_ext)

# Combine paths and ground truths into a single list
data = list(zip(image_paths, ground_truths))

# Shuffle the data
random.shuffle(data)

# Split the dataset into train, test, and eval (80:10:10)
total_count = len(data)
train_split = int(0.8 * total_count)
test_split = int(0.9 * total_count)

train_data = data[:train_split]
test_data = data[train_split:test_split]
eval_data = data[test_split:]

# Create DataFrames for each split
train_df = pd.DataFrame(train_data, columns=["image_path", "ground_truth"])
test_df = pd.DataFrame(test_data, columns=["image_path", "ground_truth"])
eval_df = pd.DataFrame(eval_data, columns=["image_path", "ground_truth"])

# Save the DataFrames as CSV files
output_dir = "/content/drive/MyDrive/GLKS/Processed Data"
os.makedirs(output_dir, exist_ok=True)

train_df.to_csv(os.path.join(output_dir, "train.csv"), index=False, header=False)
test_df.to_csv(os.path.join(output_dir, "test.csv"), index=False, header=False)
eval_df.to_csv(os.path.join(output_dir, "eval.csv"), index=False, header=False)

print(f"Train, test, and eval CSVs have been saved to {output_dir}")


Train, test, and eval CSVs have been saved to /content/drive/MyDrive/GLKS/Processed Data


In [ ]:
%cd /content/PaddleOCR/ppocr/utils

/content/PaddleOCR/ppocr/utils


In [ ]:
import csv

# Paths to the input and output files
input_file = "train.csv"   # Replace with your input CSV file path
output_file = "train.txt" # Replace with your desired output text file path

# Open the input CSV and write to the output file with a tab delimiter
with open(input_file, "r", newline="", encoding="utf-8") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=",")  # Read with ',' as the delimiter
    with open(output_file, "w", newline="", encoding="utf-8") as txt_file:
        txt_writer = csv.writer(txt_file, delimiter="\t")  # Write with tab delimiter
        for row in csv_reader:
            txt_writer.writerow(row)

print(f"CSV with ',' delimiter replaced by tabs saved to {output_file}")

CSV with ',' delimiter replaced by tabs saved to /content/drive/MyDrive/GLKS/Processed Data/train.txt


In [ ]:
!python gen_label.py --mode="rec" --input_path="train.csv" --output_label="train.txt"

Generate rec label


In [ ]:
!python gen_label.py --mode="rec" --input_path="test.csv" --output_label="test.txt"

Generate rec label


In [ ]:
!python gen_label.py --mode="rec" --input_path="eval.csv" --output_label="eval.txt"

Generate rec label


In [ ]:
%cd /content/PaddleOCR

/content/PaddleOCR


In [ ]:
!mkdir pretrain_models

In [ ]:
!wget -P ./pretrain_models/ https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_rec_train.tar

--2024-11-20 06:43:21--  https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_rec_train.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 103.235.47.176, 2409:8c04:1001:1203:0:ff:b0bb:4f27
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|103.235.47.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 204093440 (195M) [application/x-tar]
Saving to: ‘./pretrain_models/en_PP-OCRv3_rec_train.tar’

en_PP-OCRv3_rec_tra 100%[===================>] 194.64M  44.0MB/s    in 8.7s    

2024-11-20 06:43:31 (22.4 MB/s) - ‘./pretrain_models/en_PP-OCRv3_rec_train.tar’ saved [204093440/204093440]



In [ ]:
%cd pretrain_models

/content/PaddleOCR/pretrain_models


In [ ]:
!tar -xf en_PP-OCRv3_rec_train.tar && rm -rf en_PP-OCRv3_rec_train.tar

In [ ]:
%cd /content/PaddleOCR

/content/PaddleOCR


In [ ]:
!python3 tools/train.py -c /content/PaddleOCR/pretrain_models/en_PP-OCRv3_rec_train/ch_PP-OCRv3_rec_distillation.yml -o Global.pretrained_model=/content/PaddleOCR/pretrain_models/en_PP-OCRv3_rec_train/best_accuracy

[2024/11/20 07:40:53] ppocr INFO: Architecture : 
[2024/11/20 07:40:53] ppocr INFO:     Models : 
[2024/11/20 07:40:53] ppocr INFO:         Student : 
[2024/11/20 07:40:53] ppocr INFO:             Backbone : 
[2024/11/20 07:40:53] ppocr INFO:                 last_conv_stride : [1, 2]
[2024/11/20 07:40:53] ppocr INFO:                 last_pool_type : avg
[2024/11/20 07:40:53] ppocr INFO:                 name : MobileNetV1Enhance
[2024/11/20 07:40:53] ppocr INFO:                 scale : 0.5
[2024/11/20 07:40:53] ppocr INFO:             Head : 
[2024/11/20 07:40:53] ppocr INFO:                 head_list : 
[2024/11/20 07:40:53] ppocr INFO:                     CTCHead : 
[2024/11/20 07:40:53] ppocr INFO:                         Head : 
[2024/11/20 07:40:53] ppocr INFO:                             fc_decay : 1e-05
[2024/11/20 07:40:53] ppocr INFO:                         Neck : 
[2024/11/20 07:40:53] ppocr INFO:                             depth : 2
[2024/11/20 07:40:53] ppocr INFO:        